# Experiments

For a full description of the experimental setup, see the [documentation](../docs/experimental-setup.md).

## Scenario 1: Preference Criteria are Superset of Objectives (SUP)

In this first scenario, we evaluate the maximum utility of the different adaptation techniques, if the objectives do not include all preference criteria in the true preference function.

In [1]:
import experiments.user
import experiments.problem

# Get MOO problem
problem = experiments.problem.LayoutProblem(
    objectives=["neck", "shoulder"]
)

# Get utility function
get_utility = experiments.user.get_utility_function(
    objectives=["neck", "shoulder", "torso"],
    weights=1/3,
)

### Our Solver: U-NSGA-III w/ AASF

Our suggested solver uses the genetic U-NSGA-III algorithm with the AASF decomposition technique.

In [2]:
import experiments.pareto_solver
import numpy as np

def get_utilities_of_pareto_optimal_adaptations(problem, get_utility, verbose = False):
    """Get the utilities of Pareto optimal adaptations."""
    pareto_solver = experiments.pareto_solver.ParetoSolver(problem, pop=100, n_gen=100, seed=42)
    pareto_front_adaptations = pareto_solver.get_adaptations()
    utilities = [get_utility(adaptation) for adaptation in pareto_front_adaptations]
    if verbose:
        print(f"Adaptations (Ours): {len(pareto_front_adaptations)} adaptations")
        print("Utilities (Ours):", utilities)
        print("Max Utility Adaptation (Ours):", pareto_front_adaptations[np.argmax(utilities)])
        print("Max Utility (Ours):", np.max(utilities))
    return utilities

utilities_for_pareto_front_adaptations = get_utilities_of_pareto_optimal_adaptations(problem, get_utility, verbose=True)

Adaptations (Ours): 5 adaptations
Utilities (Ours): [0.8700231379722703, 0.8802704924179034, 0.8461635646145993, 0.8910355494073863, 0.8623806963310363]
Max Utility Adaptation (Ours): Layout(items=[Element(id='0', position=Position(x=-15.606332210073283, y=-13.628113330232878, z=-19.79615045376869), rotation=Rotation(x=0.43683948624245916, y=0.026316407425051758, z=0.12167580261783005, w=0.03575771701421536))])
Max Utility (Ours): 0.8910355494073863


### Baseline Solver 1: Static Weighted Sum (WS)

The first baseline solver is a static weighted sum with equal weights.

### Baseline Solver 2: Multiple Single-Objective Solvers (MSO)

The second baseline solver is a single-objective solver that is used for all objectives in the utility function.

### Baseline Solver 3: Random Solver (RND)

The third baseline solver is a random solver that returns a random adaptation.

In [3]:
import experiments.random_solver

def get_utility_of_random_adaptation(problem, get_utility, verbose = False):
    """Get the utility of a random adaptation."""
    random_solver = experiments.random_solver.RandomSolver(problem=problem, seed=42)
    random_adaptation = random_solver.get_adaptations(n_adaptations=1)
    utility = get_utility(random_adaptation)
    if verbose:
        print("Adaptation (RND):", random_adaptation)
        print("Utility (RND):", utility)
    return utility

utility_for_random_adaptation = get_utility_of_random_adaptation(problem, get_utility, verbose=True)

Adaptation (RND): Layout(items=[Element(id='0', position=Position(x=10.958241942238534, y=-2.4448624099179064, z=14.343916796455296), rotation=Rotation(x=0.6973680290593639, y=0.09417734788764953, z=0.9756223516367559, w=0.761139701990353))])
Utility (RND): 0.9106866996687881
